# Faiss RAG

## install

- [Facebook AI Similarity Search](https://www.datacamp.com/pt/blog/faiss-facebook-ai-similarity-search)
- [faiss tutorial](https://python.langchain.com/docs/integrations/vectorstores/faiss/)
- [InMemoryDocstore](https://python.langchain.com/api_reference/community/docstore/langchain_community.docstore.in_memory.InMemoryDocstore.html)
- [FAISS](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)
- [Retrieval-Augmented Generation (RAG) with LangChain and FAISS](https://medium.com/@alexrodriguesj/retrieval-augmented-generation-rag-with-langchain-and-faiss-a3997f95b551)

```bash
cd E:\programas\ia\virtual_environment && my_env_3129\Scripts\activate
uv pip install -qU langchain faiss-cpu pypdf2 openai python-dotenv langchain-ollama
```

# Building Retrieval-Augmented Generation (RAG) pipeline

## Step 1: Import libraries

In [2]:
# import os
# from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Load environment variables from .env file
#load_dotenv()

## Step 2: Create Documents

In [3]:
pdf_path = r"..\..\data\pdfs\monopoly.pdf"

loader = PyPDFLoader(file_path=pdf_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
split_documents = text_splitter.split_documents(documents)

## Step 3: Create, Save, and Load Vector Store

In [4]:
# obs: precisa do $ ollama serve

from langchain_ollama import OllamaEmbeddings
embedding_model = OllamaEmbeddings(model="nomic-embed-text") # nomic-embed-text | llama3
students_embeddings = embedding_model.embed_documents(["teste"])
print(students_embeddings)

[[0.016467305, 0.03026595, -0.17256817, 0.023868816, 0.03349949, -0.00058995635, 0.021893386, -2.7633583e-05, -0.03639252, -0.015499223, -0.026084783, 0.026844995, 0.026133616, -0.045095365, 0.005655398, -0.04929765, 0.05812456, -0.039694693, 0.007047141, -0.0011950906, 0.04336905, -0.0087991925, -0.031078953, -0.035518806, 0.13211723, 0.000746148, -0.036590494, 0.06629684, -0.03449991, -0.055991743, 0.031263728, 0.0029022687, 0.052100934, 0.035438295, 0.0149548, -0.01919272, 0.015346784, 0.049202625, 0.017465414, -0.0014161592, 0.0035245602, 0.018308444, -0.05742242, -0.033264562, 0.081448294, 0.010195554, -0.06669807, 0.05819847, -0.008660613, -0.031786375, -0.015147163, -0.033206493, 0.030374173, -0.029595485, 0.07924157, 0.02143706, 0.013627398, -0.07123822, 0.026264276, -0.02243022, -0.010141772, 0.009613517, -0.088849805, 0.09620908, 0.016503848, -0.076200716, -0.040523537, 0.083201006, 0.052718606, 0.0421865, 0.03061438, 0.048067585, 0.024585474, -0.06085921, -0.024648825, -0.01

In [5]:
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# Save the vector store
vectorstore.save_local("faiss_index")

# Load the vector store
new_vectorstore = FAISS.load_local(
    "faiss_index", embedding_model, allow_dangerous_deserialization=True
)

## Step 4: Create Retrieval QA Chat Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "llama3.2"
local_model = "deepseek-r1"
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
    versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de
    um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu
    objetivo é ajudar o usuário a superar algumas das limitações da pesquisa de similaridade 
    baseada em distância. Forneça essas perguntas alternativas separadas por quebras de linha.
    Pergunta original: {question}""",
)

In [7]:
from langchain_ollama import OllamaEmbeddings 
from langchain_chroma import Chroma

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
# vector_db = Chroma(
#     client=chroma_client,
#     collection_name=COLLECTION_NAME,
#     embedding_function=embedding_model
# )

retriever = MultiQueryRetriever.from_llm(
    new_vectorstore.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Responda à pergunta com base SOMENTE no seguinte contexto:
{context}
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
from IPython.display import display, Markdown
display(Markdown(chain.invoke("Como jogar Monopoly ?")))

<think>
Ok, eu preciso entender como jogar Monopoly com base nas instruções fornecidas. Primeiro, vejo que o jogo é dividido em várias etapas: começo com a administração do Jogo,setup das casas e finanças, jogada inicial, transação de propriedades, vender e alugar casas, derrotar ou bankruptezar, e distribuição de dinheiro. 

Em行政管理, cada jogador recebe um cartão de ingresso e um tabuleiro. Precisamos escolher um jogador como Banca. O Banca organiza o empate de Mercado, que é onde vendemos propriedades se houver empate. O Banca também administra as finanças do jogo, including Feature Gas, impostos e juros.

Em setup das finanças, cada jogador recebe $1500 distribuído de acordo com um esquema específico, incluindo bilionários, milhões, etc. Se houver mais de cinco jogadores, o Banca pode ser-limitado a ser o Banque somente.

A jogada inicial consiste emdobrar as casas e distribuir money, com um jogador escolhendo a primeira casa do Tabuleiro. Depois, cada jogador pode adicionar propriedades ao Tabuleiro, including Casas, hotéis, Immóveis com juros e Mortgações.

Em transação de propriedades, os jogadores podem vender suas casas ou imóveis por valor de mercado, considerando o Imposto de Mercado se houver. Se houver empate, o Banca determina quem ganha quais propriedades. Em finanças, se um jogador tiver dívidas maiores que seu patrimônio, ele é declarado bankrupteado e retired do jogo.

Vender e alugar casas envolve vender as casas do oponente a mais alta Oferte. Se a casinha da Rua estiver vacante, o jogador pode alugar-a paga exatamente, ou vender-a por um valor maior.

Derrotar ou Bankruptcy é determinado pelo dívida de um jogador em relação ao Banca ou os outros jogadores. Casas e hotéis estão disponíveis para compra se houver empate de Mercado.

Distribuição de dinheiro inclui premios específicos como $500, $100, etc., e o Banca mantém a reste.

Para entender melhor, talvez seja bom recorrer a um tutorial ou guide completo de jogar Monopoly, mas essas instruções abrangem as principais etapas do jogo.
</think>

Para jogar Monopoly, siga os seguintes passos organizados:

### 1. **Administração do Jogo**
   - **Jogador Inicial:** Se houver mais de cinco jogadores, escolha um jogador como Banca. O Banca organiza o empate de Mercado e administra as finanças.
   - **Setup das Finanças:** Cada jogador recebe $1500 distribuído conforme esquema: 6 bilhões ($40 milhões), 5 milhões ($105, $50, $5$) e resta para o Banca.

### 2. **Jogada Inicial**
   - **Dobrar Casas:** Jogadores, incluindo o Banca,dobram as casas do Tabuleiro.
   - **Primeira Casa:** O jogador que ganhou o empate escolhe a primeira casa da Rua para morar.

### 3. **Adicionar Propriedades**
   - Cada jogador adiciona propriedades ao Tabuleiro: Casas, hotéis, Immóveis com juros e Mortgações, seguindo os regras do jogo.

### 4. **Transação de Propriedades**
   - **Venda de Propriedades:** Jogadores podem vender suas casas ou imóveis pelo valor de mercado.
   - **Empate de Mercado:** Casas e propriedades podem ser vendidas por mais alta Oferte. Se houver empate, o Banca determina quem ganha.

### 5. **Vender e Alugar Casas**
   - Jogadores podem vender casas ao Banca ou aos opostos por valor de mercado.
   - **Aluguel de Casas:** Se a Rua está vacante, jogadores podem alugar-a paga exatamente ou vender-a por mais alto.

### 6. **Derrotar ou Bankruptcy**
   - **Banкрутêzio:** Jogadores são declarados bankrupteados se tiverem dívidas maiores que seu patrimônio. Eles se retiram do jogo.

### 7. **Distribuição de Dinheiro**
   - Dic drawer premios como $500, $100, etc., e o Banca mantém a reste.

### Resumo
- Comece com a administração do Jogo e setup das Finanças.
- Execute a jogada inicial e adicione propriedades ao Tabuleiro.
- Realize transações de propriedades, incluindo vendas e aluguel de casas.
- Monitor finanças e determinar derrotos ou Bankruptcy conforme necessário.

Se inscreva em um tutorial completo para compreender as regras completas!

In [11]:
from IPython.display import display, Markdown
display(Markdown(chain.invoke("Quanto custa uma casa e um hotel no Monopoly ?")))

<think>
Primeiro, preciso entender a estrutura do jogo de Monopoly, particularly como são calculados os custos das casas e dos hotéis. Sabemos que Monopoly é um jogo de propriedades, e cada propriedade tem seu valor base.

Segundo, lembro que o custo inicial de uma casa em Monopoly é 120 dólares, incluíndo a construção da casa herself e os terrenos adjacentes necessários. Isso contrasta com a propriedade primitiva que tem um valor mais baixo.

Em seguida, o custo de um hotel é maior. Cada hotel cobra 25 dólares de aluguel por vistoria, então um hotelindividual custaria mais. No entanto, os hotéis na Monopoly são geralmente mais caros do que as casas, especialmente se construction fee estiver incluído.

Também preciso considerar se há possibilidade de increasing o valor de uma propriedade através de imóveis adjacentes ou outros meios de ganho de valor, como a construction fee ou os efeitos da propriedade na Rua dos Homens, que pode acelerar o advancement.

Além disso, lembro que os preços dos imóveis podem variarDependendo da versão do jogo. No entanto, para um jogo tradicional, os valores de casa e hotel são estabelecidos de forma padrão.

Resumindo, a casa custa 120 dólares, incluído o edifício e os adjacentes, e o hotel custa mais, comumente estando em torno dos 350 dólares, incluído o edifício e os terrenos necessários para sua construção.
</think>

No jogo de Monopoly, um **casa** custa **120 dólares**, incluindo a construção da casa e os terrenos adjacentes necessários. Um **hotel** individual custa mais: aproximadamente **350 dólares**, incluindo o edifício e os terrenos necessários para sua construção.